<a href="https://colab.research.google.com/github/Ironsoldier353/svm_cat_dog/blob/main/Copy_of_Untitled02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"ironsoldier353","key":"9b37daec76c6fa99cde94028c72f11a1"}'}

In [2]:
import os
print(os.listdir('/content'))


['.config', 'kaggle (1).json', 'sample_data']


In [3]:
import os
import shutil

# Define paths
source_file = '/content/kaggle (1).json'
destination_dir = '/root/.kaggle/'
destination_file = os.path.join(destination_dir, 'kaggle.json')

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move and rename the file
shutil.move(source_file, destination_file)

# Set correct file permissions
os.chmod(destination_file, 0o600)


In [4]:
!pip install kaggle


In [5]:
!kaggle competitions download -c dogs-vs-cats


100% 811M/812M [00:10<00:00, 55.2MB/s]
100% 812M/812M [00:10<00:00, 79.0MB/s]


In [6]:
!unzip -q dogs-vs-cats.zip -d /content/dogs-vs-cats


In [7]:
import os

# List files and directories
os.listdir('/content/dogs-vs-cats')


['train.zip', 'test1.zip', 'sampleSubmission.csv']

In [8]:
import os
import numpy as np
import pandas as pd
import zipfile
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


In [9]:
# Define paths
main_dir_path = '/content/dogs-vs-cats'
train_zip_path = os.path.join(main_dir_path, 'train.zip')
test1_zip_path = os.path.join(main_dir_path, 'test1.zip')

# Define extraction paths
train_extract_path = os.path.join(main_dir_path, 'train')
test1_extract_path = os.path.join(main_dir_path, 'test', 'test1')

# Create extraction directories if they do not exist
os.makedirs(train_extract_path, exist_ok=True)
os.makedirs(os.path.dirname(test1_extract_path), exist_ok=True)

# Extract train.zip
if os.path.exists(train_zip_path):
    with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
        zip_ref.extractall(train_extract_path)
    print("Unzipped train.zip successfully.")
else:
    print("train.zip is missing or not found.")

# Extract test1.zip
if os.path.exists(test1_zip_path):
    with zipfile.ZipFile(test1_zip_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(test1_extract_path))
    print("Unzipped test1.zip successfully.")
else:
    print("test1.zip is missing or not found.")


Unzipped train.zip successfully.
Unzipped test1.zip successfully.


In [10]:
# Verify contents of the training directory
print("Training directory contents:")
print(os.listdir(train_extract_path))

# Verify contents of the test directory
print("Test directory contents:")
print(os.listdir(os.path.dirname(test1_extract_path)))

Training directory contents:
['train']
Test directory contents:
['test1']


In [11]:
# Define the paths to check
nested_train_dir = os.path.join(train_extract_path, 'train')
nested_test1_dir = os.path.join(main_dir_path, 'test', 'test1')

In [12]:
# Define the path to the deeper 'train' directory
deeper_train_dir = os.path.join('/content/dogs-vs-cats/train', 'train')

In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Define image size and paths
img_size = (64, 64)
train_dir = '/content/dogs-vs-cats/train/train'

# Initialize lists to hold image data and labels
data = []
labels = []

def load_images_from_directory(folder):
    """Load and preprocess images from a directory."""
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)
        img_array = img_array.flatten()  # Flatten image into a vector
        data.append(img_array)

        # Determine label from filename
        if 'cat' in filename:
            labels.append(0)  # Label 0 for cats
        elif 'dog' in filename:
            labels.append(1)  # Label 1 for dogs

# Load images and labels
load_images_from_directory(train_dir)

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)


In [2]:
# Check the shape of the training and validation sets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)


Shape of X_train: (20000, 12288)
Shape of y_train: (20000,)
Shape of X_val: (5000, 12288)
Shape of y_val: (5000,)


In [3]:
# Reduce dimensionality to make SVM training feasible
pca = PCA(n_components=300)  # Adjust the number of components based on memory and performance
X_train_reduced = pca.fit_transform(X_train)
X_val_reduced = pca.transform(X_val)

In [4]:
# Print the shapes of the reduced datasets
print(f"Shape of X_train_reduced: {X_train_reduced.shape}")
print(f"Shape of X_val_reduced: {X_val_reduced.shape}")

Shape of X_train_reduced: (20000, 300)
Shape of X_val_reduced: (5000, 300)


In [ ]:
# Train SVM model
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear'))
svm_model.fit(X_train_reduced, y_train)

Evaluation

In [19]:
# Evaluate the model
accuracy = svm_model.score(X_val_reduced, y_val)
print(f'Validation Accuracy: {accuracy:.2f}')


Validation Accuracy: 0.61


In [23]:
# Hyperparameter tuning with GridSearchCV
from sklearn.model_selection import GridSearchCV, train_test_split

# Define the parameter grid for GridSearchCV
param_grid = {
    'svc__C': [1, 10]
}

# Create a smaller sample of the training data
X_train_sample, _, y_train_sample, _ = train_test_split(X_train_reduced, y_train, train_size=0.5, random_state=42)

# Initialize GridSearchCV with n_jobs=1 to avoid parallel processing
grid_search = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=1)
grid_search.fit(X_train_sample, y_train_sample)

# Best model after grid search
best_model = grid_search.best_estimator_
print(f"Best Model: {best_model}")


Best Model: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=10, kernel='linear'))])


In [24]:
# Evaluate on the validation set
accuracy = best_model.score(X_val_reduced, y_val)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.5980


In [28]:
import os

# List contents of the directory to verify the structure
print(os.listdir('/content/dogs-vs-cats'))


['train.zip', 'train', 'test1.zip', 'test', 'sampleSubmission.csv']


In [29]:
import zipfile
import os

# Define the paths
test1_zip_path = '/content/dogs-vs-cats/test1.zip'
test_extract_path = '/content/dogs-vs-cats/test'

# Create extraction directory if it doesn't exist
os.makedirs(test_extract_path, exist_ok=True)

# Extract test1.zip
if os.path.exists(test1_zip_path):
    with zipfile.ZipFile(test1_zip_path, 'r') as zip_ref:
        zip_ref.extractall(test_extract_path)
    print("Unzipped test1.zip successfully.")
else:
    print("test1.zip is missing or not found.")


Unzipped test1.zip successfully.


In [32]:
import os

# List files in the test directory
test_dir = '/content/dogs-vs-cats/test'
print(os.listdir(test_dir))


['test1']


In [35]:
import os
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define image size (use the size you used for training)
img_size = (128, 128)

# Define the correct path to the test1 directory
test_dir = '/content/dogs-vs-cats/test/test1'

# Initialize list to hold test image data
test_data = []

def load_images_from_directory(folder):
    """Load and preprocess images from a directory."""
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):  # Ensure it's a file
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img)
            img_array = img_array.flatten()  # Flatten image into a vector
            test_data.append(img_array)

# Load test images
load_images_from_directory(test_dir)

# Convert list to numpy array
test_data = np.array(test_data)

# Apply PCA transformation to test data
# Assuming pca was fitted during training
X_test_reduced = pca.transform(test_data)

# Predict on the test set
test_predictions = best_model.predict(X_test_reduced)

# Load sample submission file
import pandas as pd
submission_df = pd.read_csv('/content/dogs-vs-cats/sampleSubmission.csv')

# Ensure to update the 'id' column with the filenames if needed
submission_df['label'] = test_predictions

# Save the submission file
submission_df.to_csv('/content/dogs-vs-cats/submission.csv', index=False)
print("Submission file created successfully.")


ValueError: X has 49152 features, but PCA is expecting 12288 features as input.